In [1]:
# Use modules from parent directory
import os
import sys
import tifffile as tiff
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000   
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from srcv2_2.models.params import get_params
from srcv2_2.models.Unet import Unet, UnetV2
from srcv2_2.visualization.visualization_utils import get_predicted_thumbnails 
from srcv2_2.utils import get_model_name

import tensorflow as tf

2024-04-08 19:11:45.595562: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 19:11:45.614161: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 19:11:45.614180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 19:11:45.614566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 19:11:45.617764: I tensorflow/core/platform/cpu_feature_guar

## Defining Params

For development purposes, use epochs=5. For final use, use epochs=20 perhaps.

In [2]:
params = get_params('U-net', 'MODIS')
params.cls = ['cloud', 'clear', 'shadow'] #'clear', 'cloud', 'shadow', 'snow', 'water'] 
#working edition: ['cloud', 'thin']
params.collapse_cls = False
params.project_path = "/home/mxh/RS-Net/"
params.activation_func='elu'
params.learning_rate = 1e-3
params.epochs = 5
params.threshold = 0.5
params.dropout = 0
params.L2reg = 0
params.decay = 0
params.num_gpus = 1
params.overlap = 40
params.modelID = "_v2_overlap40_cls-cloud-clear-shadow"
#params.norm_threshold = 3500

#tile = 'all'  # Name of one tile or 'all'
#tile = 'L2A_T32UNF_20170803T103019' 

## Training of local model

In [4]:
params.__dict__

{'modelNick': 'Unet-MOD09GA',
 'modelID': '_v2_overlap40_cls-cloud-thin-clear',
 'num_gpus': 1,
 'optimizer': 'Adam',
 'loss_func': 'binary_crossentropy',
 'activation_func': 'relu',
 'initialization': 'glorot_uniform',
 'use_batch_norm': True,
 'dropout_on_last_layer_only': True,
 'early_stopping': False,
 'reduce_lr': False,
 'save_best_only': False,
 'use_ensemble_learning': False,
 'ensemble_method': 'Bagging',
 'learning_rate': 0.001,
 'dropout': 0,
 'L1reg': 0.0,
 'L2reg': 0,
 'L1L2reg': 0.0,
 'decay': 0,
 'batch_norm_momentum': 0.7,
 'threshold': 0.5,
 'patch_size': 256,
 'overlap': 40,
 'overlap_train_set': 0,
 'batch_size': 40,
 'steps_per_epoch': None,
 'epochs': 5,
 'norm_method': 'enhance_contrast',
 'norm_threshold': 3500,
 'cls': ['cloud', 'thin', 'clear'],
 'collapse_cls': False,
 'affine_transformation': True,
 'brightness_augmentation': False,
 'bands': [1, 2, 3, 4, 5, 6, 7],
 'project_path': '/home/mxh/RS-Net/',
 'toa_path': 'data/processed/Biome_TOA/',
 'data_path': 

In [3]:
# training local (notebook) model
local_model = UnetV2(params)
local_model.get_config()

2024-04-08 19:11:51.803153: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-08 19:11:51.822611: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-08 19:11:51.822707: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

No weights found


{'seed': 1,
 'params': <srcv2_2.models.params.HParams at 0x7b9ac9566110>,
 'training_params': None,
 'n_cls': 3,
 'n_bands': 7}

In [4]:
local_model.train()

/home/mxh/anaconda3/envs/tf2+gpu/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


------------------------------------------
Start training:
Epoch 1/5


2024-04-08 19:11:54.488872: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8800


477/477 [==============================] - 134s 244ms/step - loss: 0.0020 - binary_crossentropy: 0.0020 - jaccard_coef_loss: 0.2817 - jaccard_coef: 0.9904 - jaccard_coef_thresholded: 0.9943 - accuracy: 0.9911 - val_loss: 0.0000e+00 - val_binary_crossentropy: 0.0000e+00 - val_jaccard_coef_loss: 0.0000e+00 - val_jaccard_coef: 1.0000 - val_jaccard_coef_thresholded: 1.0000 - val_accuracy: 1.0000
Epoch 2/5
477/477 [==============================] - 112s 233ms/step - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - jaccard_coef_loss: 0.0000e+00 - jaccard_coef: 1.0000 - jaccard_coef_thresholded: 1.0000 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_binary_crossentropy: 0.0000e+00 - val_jaccard_coef_loss: 0.0000e+00 - val_jaccard_coef: 1.0000 - val_jaccard_coef_thresholded: 1.0000 - val_accuracy: 1.0000
Epoch 3/5
477/477 [==============================] - 115s 241ms/step - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - jaccard_coef_loss: 0.0000e+00 - jaccard_coef: 1.0000 - jaccard_c

In [7]:
get_model_name(params)

'Unet_MODIS__v2_overlap40_cls-cloud-thin-clear'

In [5]:
# check if training_params are saved
local_model.get_config()["params"].__dict__

{'modelNick': 'Unet-MOD09GA',
 'modelID': '_v2_overlap40_cls-cloud-thin-clear',
 'num_gpus': 1,
 'optimizer': 'Adam',
 'loss_func': 'binary_crossentropy',
 'activation_func': 'relu',
 'initialization': 'glorot_uniform',
 'use_batch_norm': True,
 'dropout_on_last_layer_only': True,
 'early_stopping': False,
 'reduce_lr': False,
 'save_best_only': False,
 'use_ensemble_learning': False,
 'ensemble_method': 'Bagging',
 'learning_rate': 0.001,
 'dropout': 0,
 'L1reg': 0.0,
 'L2reg': 0,
 'L1L2reg': 0.0,
 'decay': 0,
 'batch_norm_momentum': 0.7,
 'threshold': 0.5,
 'patch_size': 256,
 'overlap': 40,
 'overlap_train_set': 0,
 'batch_size': 40,
 'steps_per_epoch': None,
 'epochs': 5,
 'norm_method': 'enhance_contrast',
 'norm_threshold': 3500,
 'cls': ['cloud', 'thin', 'clear'],
 'collapse_cls': False,
 'affine_transformation': True,
 'brightness_augmentation': False,
 'bands': [1, 2, 3, 4, 5, 6, 7],
 'project_path': '/home/mxh/RS-Net/',
 'toa_path': 'data/processed/Biome_TOA/',
 'data_path': 